In [1]:
import torch
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

/home/mohammadhasan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import torchvision 
from torchvision import transforms

image_path = '../images/'
transform = transforms.Compose([
    transforms.ToTensor()
])

mnist_train_dataset = torchvision.datasets.MNIST(
    root=image_path, train=True, transform=transform, download=False
)

mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path, train=False, transform=transform, download=False
)


In [7]:
from torch.utils.data import DataLoader

batch_size = 64
torch.manual_seed(1)

train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

In [12]:
import torch.nn as nn 

hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]

all_layer = [nn.Flatten()]

for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layer.append(layer)
    all_layer.append(nn.ReLU())
    input_size = hidden_unit
    
all_layer.append(nn.Linear(hidden_units[-1], 10))

model = nn.Sequential(*all_layer)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)

num_epochs = 20

for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = ( torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()

    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch} Accuracy ', f'{accuracy_hist_train:.4f}')

Epoch 0 Accuracy  0.6494
Epoch 1 Accuracy  0.8688
Epoch 2 Accuracy  0.9009
Epoch 3 Accuracy  0.9201
Epoch 4 Accuracy  0.9266
Epoch 5 Accuracy  0.9330
Epoch 6 Accuracy  0.9400
Epoch 7 Accuracy  0.9421
Epoch 8 Accuracy  0.9462
Epoch 9 Accuracy  0.9506
Epoch 10 Accuracy  0.9561
Epoch 11 Accuracy  0.9597
Epoch 12 Accuracy  0.9641
Epoch 13 Accuracy  0.9645
Epoch 14 Accuracy  0.9651
Epoch 15 Accuracy  0.9701
Epoch 16 Accuracy  0.9722
Epoch 17 Accuracy  0.9740
Epoch 18 Accuracy  0.9755
Epoch 19 Accuracy  0.9782


In [16]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = ( torch.argmax(pred, dim=1) == mnist_test_dataset.targets  ).float()
print(f'Test accuracy: {is_correct.mean():.4f}')

Test accuracy: 0.9819
